In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#___________________________________________________

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn import metrics
from sklearn.tree import  DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
import pandas_profiling

# ------------------------------------------------ extrelty -----------------------------------

#### Description of the activity

- **EXRELACTCD** - Activity code of the donor
- **EXRELACTDE** - Activity desc of the donor 

In [3]:
extrelty = pd.read_sas('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/1 - Data for Churn Modeling/extrelty.sas7bdat',encoding= 'utf-8')
extrelty.head()

,EXRELACTCD,EXRELACTDE
0,FP,CommitmentDo
1,PS,Prospect
2,PF,Prosp Fundra
3,PP,Prosp Promot
4,GR,Grantor


In [15]:
extrelty.shape

(26, 2)

# ------------------------------------------- extrel -------------------------------------------

#### All the donors of the organization

**Extrelno** - Unique identifier of each donor

**Extrelstdt** Start date of the relationship

**Exreldaten** - End date of the relationship (Missing: not ended)

In [4]:
extrel = pd.read_sas('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/1 - Data for Churn Modeling/extrel.sas7bdat')
extrel.head(2)

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,b'0026414',b'CT',1995-02-02,NaT
1,b'0026419',b'FP',1992-03-01,NaT


#### Decoding the utf-8 Encoded values 

In [5]:
extrel[['EXTRELNO','EXRELACTCD']] = extrel[['EXTRELNO','EXRELACTCD']].apply( lambda x:x.str.decode('utf-8'))

In [445]:
extrel.head(2)

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,0026414,CT,1995-02-02,NaT
1,0026419,FP,1992-03-01,NaT


In [446]:
## Only "EXRELDATEN" has Null values coz it signifies if a user is active or not
extrel.describe().T

,count,unique,top,freq,first,last
EXTRELNO,26689,18704,0038595,6,NaN,NaN
EXRELACTCD,26689,17,FP,11442,NaN,NaN
EXTRELSTDT,26689,2877,1994-11-29 00:00:00,1039,1989-11-01 00:00:00,2007-02-01 00:00:00
EXRELDATEN,10168,2567,1994-12-07 00:00:00,58,1927-10-31 00:00:00,2007-01-25 00:00:00


In [8]:
# extrel.EXRELDATEN.value_counts()

- Unique users that have a history after year 2003

- Spillitng the data after 2003 yr will result in a major loss of data [18704 vs 2277]

In [6]:
data = extrel[extrel.EXRELDATEN >= pd.to_datetime('2003-01-01')]

# Using Agg function without groupby
data.EXTRELNO.agg(['nunique','count'])

nunique    2127
count      2277
Name: EXTRELNO, dtype: int64

#### Activity counts

In [7]:
extrel.EXRELACTCD.value_counts().to_frame().T

,FP,PS,CT,MR,TR,AP,SP,PF,OR,PV,PO,GR,OI,PP,RV,PT,BM
EXRELACTCD,11442,10677,3325,1110,45,37,13,9,7,5,5,4,4,2,2,1,1


#### How many activities per user

In [8]:
extrel[extrel.EXTRELNO == '0038595']

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
3311,0038595,FP,1994-03-01,2001-09-17
3312,0038595,TR,1995-05-29,2004-03-16
3313,0038595,CT,2001-04-02,2001-09-19
3314,0038595,FP,2001-04-01,2002-05-07
3315,0038595,FP,2001-11-01,2002-12-10
3316,0038595,FP,2006-05-01,2007-01-12


- Number of **Activities per user**

In [9]:
extrel.groupby(['EXTRELNO']).count().sort_values(['EXRELACTCD'],ascending = False).head()

,EXRELACTCD,EXTRELSTDT,EXRELDATEN
EXTRELNO,,,
0038595,6,6,6
0175871,5,5,4
0221339,5,5,5
0211269,5,5,3
0158504,5,5,3


In [30]:
extrel.shape

(26689, 4)

- We have more than 50% values in EXRELDATEN as Null,thats becoz those members have no last_date & are thus **Active**


- We are now replacing the NAN(currently present) from data generation date(2007-02-02) & we would find the differnce of it from the start_date so to know how much time the person has stayed or is staying from.


- based on the difference found(time a user has stayed) we would segregate the date into Training/Testing data.

#### Finding the time for which the person has remained active

In [310]:
# extrel['Last_date'] = np.where(extrel.EXRELDATEN.isnull(),'2007-02-02',extrel.EXRELDATEN.tolist())

# extrel['Activity_period_Yrs'] =  round((extrel.Last_date - extrel.EXTRELSTDT).dt.days/365)

# extrel.head(3)

# extrel[extrel.Activity_period_Yrs < 4].shape

# ------------------------------------- Communication -----------------------------------

In [10]:
communication = pd.read_sas('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/1 - Data for Churn Modeling/communication.sas7bdat')

#### Decoding the encoded columns

- **Contid** - Unique identifier for each communication 
- **Mediumcode** - Medium of the contact (CI is unknown)
- **Mntopcode** - Main topic code of the contact
- **Classcode** - Class of the contact
- **Extrelno** - Unique identifier for each donor
- **Contdirec** - Direction of the communication
            I Incoming
            O Outgoing
- **Contdate** - Date of the contact
- **Description**









In [11]:
communication[communication.columns.difference(['CONTDATE'])] = communication[communication.columns.difference(['CONTDATE'])].apply(lambda x:x.str.decode('utf-8'))

In [171]:
communication.shape

(763569, 7)

In [29]:
communication.head()

,CONTID,MEDIUMCODE,MNTOPCODE,CLASCODE,EXTRELNO,CONTDIREC,CONTDATE
0,000405402,CI,CTS,CT,0000020,O,1994-05-09
1,000430904,CI,CTS,CT,0000020,O,1994-06-14
2,000592732,CI,CTS,CT,0000020,O,1994-10-21
3,000593375,CI,CTS,CT,0000020,O,1994-10-21
4,000618514,CI,CTS,CT,0000020,O,1994-11-28


In [22]:
communication.describe().T

,count,unique,top,freq,first,last
CONTID,763569,763328,005060419,2,NaN,NaN
MEDIUMCODE,763569,15,CI,446595,NaN,NaN
MNTOPCODE,763569,24,CTS,453839,NaN,NaN
CLASCODE,763569,9,CT,459032,NaN,NaN
EXTRELNO,763569,21932,0181194,1008,NaN,NaN
CONTDIREC,763569,2,O,733423,NaN,NaN
CONTDATE,763569,3290,2001-11-20 00:00:00,10760,1993-11-02 00:00:00,2007-01-25 00:00:00


In [13]:
# 241 duplicate valeus for communication ID 

# In 241 values,there were cases where for a given CONTID we had different ExtrelNO, so we wont be dropping them 
communication[communication.CONTID.duplicated()].CONTID.shape[0]

241

#### Dropping the duplicate records(where all values are same)

In [14]:
communication = communication[-communication.duplicated()]

In [15]:
print('Duplicate CONTID(commnunication ID) : ',communication[communication.CONTID.duplicated()].CONTID.shape[0])

# For the same CONTID we have differenr ExtrelNO!!
communication[communication.CONTID.duplicated()].head(2)

Duplicate CONTID(commnunication ID) :  53


,CONTID,MEDIUMCODE,MNTOPCODE,CLASCODE,EXTRELNO,CONTDIREC,CONTDATE
62982,004832516,CI,CTS,CT,0006831,O,2005-05-18
294454,004833236,CI,CTS,CT,0028744,O,2005-05-18


- One of the ID getting repeated

- Remove duplicate records!!!

#### Yearwise Users data[ for selecting the independent window]

In [16]:
communication.groupby(communication.CONTDATE.dt.year).agg({'EXTRELNO':['nunique','count']}).T

CONTDATE          1993   1994   1995   1996   1997   1998   1999   2000  \
EXTRELNO nunique   115   9350  10805  10327  10357  10266  10942  11479   
         count     115  38892  83677  60707  49905  61306  55081  58265   

CONTDATE           2001   2002   2003   2004   2005   2006  2007  
EXTRELNO nunique  11393  11514  11129  10923  10940  10921  2058  
         count    63407  73543  70048  50419  47259  47865  2892

#### Multiple communications made by a user

In [17]:
communication.groupby('EXTRELNO').agg({'CONTID':'count','CONTDATE': ['min','max'],'CONTDIREC':'nunique','CLASCODE':'nunique','MEDIUMCODE':'nunique','MNTOPCODE':'nunique'}).head(2)

CONTID   CONTDATE            CONTDIREC CLASCODE MEDIUMCODE MNTOPCODE
          count        min        max   nunique  nunique    nunique   nunique
EXTRELNO                                                                     
0000020      53 1994-05-09 2006-08-10         1        4          4         4
0000035      61 1994-06-15 2006-10-03         2        5          4         8

# ----------------------------------------- comcls --------------------------------------------

#### comcls - class of communication (Purpose of the communication)

- CLASCODE - contact class code 

In [18]:
comcls = pd.read_sas('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/1 - Data for Churn Modeling/comclas.sas7bdat',encoding= 'utf-8')
comcls

,CLASCODE,CLASDESC
0,CT,CTS-item
1,RI,Request for information
2,CM,Complaint
3,TA,Threath / Accusation
4,AK,Acknowledgement
5,RM,Reminder
6,ID,Information delivery
7,AN,Answer


# ------------------------------------ commaint ------------------------------------------

#### Description of main topic code

In [19]:
commaint = pd.read_sas('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/1 - Data for Churn Modeling/commaint.sas7bdat',encoding='utf-8')

commaint.head(2)

,MNTOPCODE,MNTOPDESC
0,CAN,Cancellations/replacements
1,DIS,Discontinuations


In [20]:
commaint.T

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
MNTOPCODE,CAN,DIS,MKT,GEN,NEW,PRO,MFC,SPC,INF,ENR,...,VOL,CTS,SPO,PRA,ALL,TVS,TAX,TLT,ACT,WEB
MNTOPDESC,Cancellations/replacements,Discontinuations,Marketing - Enquiries,General,New relations,Projects,"Mail met FC, klachten en info",Special Forms,information about Plan,Enrollment,...,Volunteers,Communication Tracking System,Contacts for sponsoring,Public Relations activities,General,TV-Show,Tax forms,Translation,Action,Web related Actions: E-commerce


In [21]:
commaint[(commaint.MNTOPCODE == 'CAN')|(commaint.MNTOPCODE == 'DIS')]

,MNTOPCODE,MNTOPDESC
0,CAN,Cancellations/replacements
1,DIS,Discontinuations


# ----------------------------------- commedium -------------------------------------------

#### Description of medium type

In [22]:
commedium = pd.read_sas('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/1 - Data for Churn Modeling/commediu.sas7bdat',encoding='utf-8')
commedium

,MEDIUMCODE,MEDIUMDESC
0,FF,Face to Face contact
1,FX,Fax
2,TX,Telex
3,EM,Electronic Mail
4,TE,Telephone call
5,ES,Express mail
6,AM,Answering machine
7,RM,Regular mail (CTS-items only)
8,AF,Application form
9,LE,Letters


# ---------------------------------------- Nameaddr ------------------------------------------

#### Sociodemographical information

**Name1title** - Title to address someone

**Languagecode** - Preferred mailing language



In [23]:
Nameaddr =  pd.read_sas('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/1 - Data for Churn Modeling/nameaddr.sas7bdat')

In [206]:
Nameaddr.dtypes

EXTRELNO      object
NAME1TITLE    object
POSTCODE      object
LANGUACODE    object
dtype: object

In [24]:
Nameaddr['NAME1TITLE']=Nameaddr.iloc[:,1].str.decode("utf-8")
Nameaddr['EXTRELNO']=Nameaddr.iloc[:,0].str.decode("utf-8")
Nameaddr['POSTCODE']=Nameaddr.iloc[:,2].str.decode("utf-8")
Nameaddr['LANGUACODE']=Nameaddr.iloc[:,3].str.decode("utf-8")

In [209]:
Nameaddr.head(2)

,EXTRELNO,NAME1TITLE,POSTCODE,LANGUACODE
0,0000019,HR,2500,NL
1,0000020,HR,9550,NL


# --------------------------------Payhistory ----------------------------------------------


#### Payhistory: Paymenthistory of each donor

**Pid** - Unique identifier for each payment

**Pdate**- Date of payment

**Pamt** - Amount of payment


**Paytypecd** 

Paytype

O Bank transfer

D Permanent order

E Own initiative

X Unknown

**Status**

Status of payment

OK Normal/Real payment

CO Correction (internal)

RF RF (Refund)

RC Recall

In [25]:
payhistory = pd.read_sas('/home/rohan/Documents/ROHAN_DA/ALABS/___STATS___ -  cls 10 onwards/Class_20 Case_study_ML/Case_study/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/2. NGO-FUND RAISING ATTRITION- CLASSIFICATION/1 - Data for Churn Modeling/payhistory.sas7bdat')

In [26]:
payhistory.head(2)

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38.0,1993-12-31,9.57,b'0000019',b'X',b'CO'
1,39.0,1993-12-31,41.32,b'0000020',b'X',b'CO'


#### Decoding

In [27]:
payhistory[['STATUS','PAYTYPECD','EXTRELNO']] = payhistory[['STATUS','PAYTYPECD','EXTRELNO']].apply(lambda x:x.str.decode('utf-8'))

In [28]:
payhistory.head()

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38.0,1993-12-31,9.57,0000019,X,CO
1,39.0,1993-12-31,41.32,0000020,X,CO
2,40.0,1993-12-31,13.10,0000020,X,CO
3,54091.0,1994-01-01,2.02,0000020,X,CO
4,104480.0,1994-05-19,0.50,0000020,D,OK


In [29]:
# We dont have to take care of the sign of PAMT as it is already taken care of based on the status

payhistory.groupby('STATUS').sum()

,PID,PAMT
STATUS,,
CO,6.778372e+10,-5.595999e+05
OK,3.109679e+12,3.211546e+07
RC,2.364133e+10,-4.289997e+05
RF,3.994299e+08,-2.074822e+04


In [30]:
payhistory.describe().T

,count,mean,std,min,25%,50%,75%,max
PID,1205720.0,2.655267e+06,1.480799e+06,38.00,1377467.50,2709602.00,3946880.25,5101978.00
PAMT,1205720.0,2.579925e+01,3.725114e+01,-2436.96,20.33,20.33,23.00,2436.96


#### Yearwise User data

In [32]:
payhistory.groupby(payhistory.PDATE.dt.year).agg({'EXTRELNO':['nunique','count']}).T

PDATE              1993   1994   1995   1996   1997   1998   1999   2000  \
EXTRELNO nunique   7576   7716   9057   9262   9295   9355   9557  10342   
         count    11175  55786  83269  87579  87721  88727  90099  97009   

PDATE               2001    2002   2003   2004   2005   2006  2007  
EXTRELNO nunique   10164   10138   9942  10051  10200  10114  7192  
         count    101807  103670  97607  98235  98434  97088  7514

#### How many times does a user made any payments(credit & debit)

In [33]:
payhistory.groupby('EXTRELNO').agg({'PID':['nunique','count'],'PDATE':'max','PAMT':['mean','sum'],'PAYTYPECD':'nunique'}).head(4)

PID            PDATE       PAMT          PAYTYPECD
         nunique count        max       mean      sum   nunique
EXTRELNO                                                       
0000019        1     1 1993-12-31   9.570000     9.57         1
0000020      158   158 2007-01-17  20.107405  3176.97         2
0000035      155   155 2007-01-17  20.441290  3168.40         2
0000042       85    85 2001-01-23  18.153294  1543.03         2

# ------------------------- Preparing Independent ---------------------------------


####  -----------------------------------------Independent window - 2005 & Before  

- Independent window gives us the data for Independent variable such as Recency|Frequency|Monetary_Value

**Possibility 1:**

**To have Timebased Training & Testing datasets**

This is not the best approach because it will lead to a lot datapoints present in both training & testing sets

### ------------------------- Independent data prep [Monetary Data]---------------------------------

In [34]:
ind_data = payhistory[payhistory.PDATE < pd.to_datetime('2006-01-01')]

print('ind_dat shape: ',ind_data.shape[0],'|| Total_data before date filter shape: ',payhistory.shape[0])

ind_dat shape:  1101118 || Total_data before date filter shape:  1205720


#### Making the Monetary Dataset for RFM Model

In [35]:
ind_data = ind_data.groupby('EXTRELNO').agg({'PDATE':'max','PAMT':['sum','mean'],'PID':'count'})

ind_data.head()

PDATE     PAMT              PID
                max      sum       mean count
EXTRELNO                                     
0000019  1993-12-31     9.57   9.570000     1
0000020  2005-12-14  2875.97  19.834276   145
0000035  2005-12-14  2867.40  20.192958   142
0000042  2001-01-23  1543.03  18.153294    85
0000043  2003-07-14  4373.24  19.097118   229

In [37]:
## Dropping the multiindex_level

#ind_data.columns =ind_data.columns.droplevel()

In [38]:
## Recency

ind_data['M_recency'] = pd.to_datetime('2006-01-01') - ind_data['max'] 

In [39]:
ind_data.head()

,max,sum,mean,count,M_recency
EXTRELNO,,,,,
0000019,1993-12-31,9.57,9.570000,1,4384 days
0000020,2005-12-14,2875.97,19.834276,145,18 days
0000035,2005-12-14,2867.40,20.192958,142,18 days
0000042,2001-01-23,1543.03,18.153294,85,1804 days
0000043,2003-07-14,4373.24,19.097118,229,902 days


In [41]:
#### Finalizing the dataset

ind_data.rename(columns={'sum':'Monetary_sum','mean':'Monetary_mean','count':'M_frequecy'},inplace=True)
#ind_data.drop(columns= 'max',inplace=True)

In [44]:
## Converting into only day level

#ind_data.M_recency = ind_data.M_recency.dt.days

#### Final Independent_data

In [45]:
ind_data.head(2)

,Monetary_sum,Monetary_mean,M_frequecy,M_recency
EXTRELNO,,,,
0000019,9.57,9.570000,1,4384
0000020,2875.97,19.834276,145,18


In [46]:
ind_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Monetary_sum,15945.0,1757.861805,1359.136785,-421.17,633.730000,1717.450000,2795.780000,52582.79
Monetary_mean,15945.0,37.667652,43.856001,-138.00,19.461538,21.179545,35.694146,600.00
M_frequecy,15945.0,69.057259,64.396949,1.00,18.000000,52.000000,127.000000,2593.00
M_recency,15945.0,975.284164,1462.214801,2.00,18.000000,45.000000,1742.000000,4384.00


In [47]:
## No. of Null enteries in columns 
ind_data.isnull().sum()

Monetary_sum     0
Monetary_mean    0
M_frequecy       0
M_recency        0
dtype: int64

In [48]:
## Verified that data is doner level
ind_data.index.nunique()

15945

### ------------------------- Independent data prep [Communication Data]---------------------------------

In [49]:
communication.head()

,CONTID,MEDIUMCODE,MNTOPCODE,CLASCODE,EXTRELNO,CONTDIREC,CONTDATE
0,000405402,CI,CTS,CT,0000020,O,1994-05-09
1,000430904,CI,CTS,CT,0000020,O,1994-06-14
2,000592732,CI,CTS,CT,0000020,O,1994-10-21
3,000593375,CI,CTS,CT,0000020,O,1994-10-21
4,000618514,CI,CTS,CT,0000020,O,1994-11-28


In [50]:
ind_data_c = communication[communication.CONTDATE < pd.to_datetime('2006-01-01')]

print('ind_dat_c shape: ',ind_data_c.shape[0],'|| Total_data before date filter shape: ',communication.shape[0])

ind_dat_c shape:  712624 || Total_data before date filter shape:  763381


In [51]:
ind_data_c.CLASCODE.value_counts()

CT    426783
ID    189819
AN     35896
RI     22449
AK     16085
PM     10619
RM      8557
CM      2292
NS       124
Name: CLASCODE, dtype: int64

In [52]:
def complain(x):
    if x == 'CM':
        return 1
    else:
        return 0

In [53]:
x = ind_data_c.CLASCODE.apply(complain)
ind_data_c['Complain'] = x.tolist()

y = ind_data_c.groupby('EXTRELNO').sum()

/home/rohan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [54]:
y.shape

(21373, 1)

In [55]:
ind_data_c = ind_data_c.groupby('EXTRELNO').agg({'CONTID':'count','CONTDATE':'max'})

In [56]:
## Recency

ind_data_c['C_Recency'] = pd.to_datetime('2006-01-01') - ind_data_c.CONTDATE
ind_data_c['C_Recency'] = ind_data_c.C_Recency.dt.days

In [57]:
ind_data_c.head()

,CONTID,CONTDATE,C_Recency
EXTRELNO,,,
0000020,50,2005-12-08,24
0000035,57,2005-11-03,59
0000042,36,2001-03-29,1739
0000043,112,2004-01-23,709
0000060,36,2000-02-12,2150


#### Joining the Complaint column 

- Logic Used

- if the sum of a user's complaint > 0 then he has made a complaint || if its 0 then he has not

In [58]:
ind_data_c = pd.merge(ind_data_c,y,right_index=True,left_index=True)

In [59]:
ind_data_c['Complain'] = ind_data_c.Complain.apply(lambda x: 1 if x > 0 else 0)

In [61]:
#ind_data_c.drop(columns='CONTDATE',inplace=True)

In [62]:
## renaing the columns

ind_data_c.rename(columns={'CONTID':'C_Frequency'},inplace=True)

In [63]:
ind_data_c.head(2)

,C_Frequency,C_Recency,Complain
EXTRELNO,,,
0000020,50,24,0
0000035,57,59,0


In [64]:
ind_data_c.shape

(21373, 3)

##  Joining the independent data for both Monetary & Communication

In [65]:
ind_data.head()

,Monetary_sum,Monetary_mean,M_frequecy,M_recency
EXTRELNO,,,,
0000019,9.57,9.570000,1,4384
0000020,2875.97,19.834276,145,18
0000035,2867.40,20.192958,142,18
0000042,1543.03,18.153294,85,1804
0000043,4373.24,19.097118,229,902


In [66]:
ind_data_c.head()

,C_Frequency,C_Recency,Complain
EXTRELNO,,,
0000020,50,24,0
0000035,57,59,0
0000042,36,1739,0
0000043,112,709,0
0000060,36,2150,0


#### Checking the overlapping, mismatch ExtrelNO in both datasets

In [67]:
print('Monetary || Communication:',ind_data.index.intersection(ind_data_c.index).shape[0])

Monetary || Communication: 14958


In [68]:
print('Monetary - Communication:',ind_data.index.difference(ind_data_c.index).shape[0])
print('Communication - Monetary:',ind_data_c.index.difference(ind_data.index).shape[0])

Monetary - Communication: 987
Communication - Monetary: 6415


#### Thus we would be woulking with 14598 common values

In [69]:
ind_data_f = pd.merge(ind_data,ind_data_c,left_index=True,right_index=True) 

#### Final Independent dataset

In [70]:
ind_data_f.head(2)

,Monetary_sum,Monetary_mean,M_frequecy,M_recency,C_Frequency,C_Recency,Complain
EXTRELNO,,,,,,,
0000020,2875.97,19.834276,145,18,50,24,0
0000035,2867.40,20.192958,142,18,57,59,0


- The data might require clipping but trying to build model without tweaking the data

In [71]:
ind_data_f.describe(percentiles=[0.01,.05,.25,.5,.75,.95,.99]).T

,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max
Monetary_sum,14958.0,1873.139703,1324.541005,-322.0,6.550,120.938500,874.00000,1878.180000,2801.480000,2872.459000,5634.493800,52582.79
Monetary_mean,14958.0,39.487852,44.586251,-138.0,1.765,13.691393,19.66059,21.333485,41.161082,137.377592,252.679091,600.00
M_frequecy,14958.0,73.514240,64.028301,1.0,1.000,5.000000,22.00000,58.000000,131.000000,145.000000,266.860000,2593.00
M_recency,14958.0,750.391897,1209.143684,2.0,4.000,18.000000,18.00000,30.000000,1238.000000,3616.000000,4256.150000,4384.00
C_Frequency,14958.0,46.808129,31.883143,1.0,3.570,8.000000,26.00000,47.000000,58.000000,92.000000,186.860000,768.00
C_Recency,14958.0,712.249632,1077.038412,2.0,4.000,12.000000,58.00000,160.000000,1065.000000,3225.000000,3963.000000,4265.00
Complain,14958.0,0.101952,0.302595,0.0,0.000,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000,1.00


In [72]:
ind_data_f.isnull().sum()

Monetary_sum     0
Monetary_mean    0
M_frequecy       0
M_recency        0
C_Frequency      0
C_Recency        0
Complain         0
dtype: int64

# ----------------------------- Preparing the dependent data -----------------------------
#### ------------------------------------------------- Dependent window 2006-01-01 to 2006 -12-31

- In dependent window we see the status of the user, whether he has churned or not!

- The dependent window thus gives us our dependent variable **(Churn or Not)**


- **HoW TO CHOOSE DEPENDENT WINDOW**

- For specified window, we would take users

- (who have not churned OR who have churned in year 2006)|| Given that the user started before 2006 Yr

In [73]:
print('shape of dependent data:',extrel.shape)
extrel.head()

shape of dependent data: (26689, 4)


,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,0026414,CT,1995-02-02,NaT
1,0026419,FP,1992-03-01,NaT
2,0026424,FP,1992-03-01,1996-12-24
3,0026430,FP,1992-03-01,2006-01-31
4,0026430,CT,1997-03-07,NaT


In [74]:
dep_data = extrel[((extrel.EXRELDATEN >= '2006-01-01') & (extrel.EXRELDATEN < '2007-01-01')) | extrel.EXRELDATEN.isnull() == True]

In [75]:
dep_data = dep_data[dep_data.EXTRELSTDT < '2006-01-01']

In [76]:
dep_data.count()

EXTRELNO      16389
EXRELACTCD    16389
EXTRELSTDT    16389
EXRELDATEN      508
dtype: int64

In [77]:
dep_data.EXTRELNO.nunique()

14752

- setting EXTRELNO as index

In [78]:
dep_data= dep_data.set_index(keys='EXTRELNO')

In [ ]:
#### Mapping churn as 1 and not churn as 0

In [79]:
dep_data['churn'] = np.where(dep_data.EXRELDATEN.isnull(),0,1)

In [80]:
dep_data.head()

,EXRELACTCD,EXTRELSTDT,EXRELDATEN,churn
EXTRELNO,,,,
0026414,CT,1995-02-02,NaT,0
0026419,FP,1992-03-01,NaT,0
0026430,FP,1992-03-01,2006-01-31,1
0026430,CT,1997-03-07,NaT,0
0026431,FP,1992-03-01,NaT,0


In [81]:
dep_data.churn.value_counts()

0    15881
1      508
Name: churn, dtype: int64

#### Between independent & dependent data we are getting only ~9000 values for making model

In [82]:
ind_data_f.index.intersection(dep_data.index) 

Index(['0026414', '0026419', '0026430', '0026430', '0026431', '0026431',
       '0026443', '0026444', '0026446', '0026447',
       ...
       '0240800', '0240808', '0240809', '0240867', '0240890', '0240915',
       '0240916', '0240920', '0240936', '0240937'],
      dtype='object', name='EXTRELNO', length=9323)

- Only 9000 matching values can be explained by looking at the min values for all communicaton|payhistory(used in independent) vs extrel(used in dependent)

In [83]:
print(communication[communication.EXTRELNO.astype(int)< 28000].shape)

extrel.EXTRELNO.min(),communication.EXTRELNO.min(),payhistory.EXTRELNO.min()

(285597, 7)


('0026414', '0000020', '0000019')

#### Final Data

In [84]:
Final_data = pd.merge(ind_data_f,dep_data,left_index=True,right_index=True)
Final_data.head(2)

,Monetary_sum,Monetary_mean,M_frequecy,M_recency,C_Frequency,C_Recency,Complain,EXRELACTCD,EXTRELSTDT,EXRELDATEN,churn
EXTRELNO,,,,,,,,,,,
0026414,2811.80,19.662937,143,18,53,332,0,CT,1995-02-02,NaT,0
0026419,2804.05,43.813281,64,89,56,68,0,FP,1992-03-01,NaT,0


In [86]:
#Final_data.drop(columns=['EXRELACTCD','EXTRELSTDT','EXRELDATEN'],inplace=True)

In [87]:
Final_data.head(2)

,Monetary_sum,Monetary_mean,M_frequecy,M_recency,C_Frequency,C_Recency,Complain,churn
EXTRELNO,,,,,,,,
0026414,2811.80,19.662937,143,18,53,332,0,0
0026419,2804.05,43.813281,64,89,56,68,0,0


In [110]:
Final_data.churn.value_counts()

0    8857
1     466
Name: churn, dtype: int64

In [89]:
churned = Final_data[Final_data.churn == 1]
Active = Final_data[Final_data.churn == 0]

####  ----------------------------

In [115]:
X = Final_data[Final_data.columns.difference(['churn'])]
Y = Final_data.churn

In [117]:
X.shape,Y.shape

((9323, 7), (9323,))

In [118]:
X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.2,random_state=0)

#### Using smote analysis for Oversampling

In [119]:
from imblearn.over_sampling import SMOTE

sme = SMOTE(random_state= 123,k_neighbors=3,ratio = 1.0,)

In [129]:
x_train_res, y_train_res = sme.fit_sample(X_train,y_train)

#### Mixing - Kind of Undersampling

In [234]:
# mix=pd.concat([Active.sample(n=1200),churned.sample(n=450)])

# mix.churn.value_counts()

# X = mix[Final_data.columns.difference(['churn'])]
# Y = mix.churn

# X.shape,Y.shape

# X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.2,random_state=0)

# Modelling

In [785]:
from sklearn.feature_selection import RFE

#### RandomForest

In [235]:
rf_gscv = GridSearchCV(RandomForestClassifier(oob_score=True),cv=5,verbose=True,scoring='accuracy',n_jobs=-1,
             param_grid={'n_estimators':[100,200,300,400,500,600]})

rf_gscv.fit(x_train_res, y_train_res)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500, 600]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=True)

In [236]:
rf_gscv.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=None,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [237]:
y_pred = rf_gscv.best_estimator_.predict(X_test)

In [238]:
metrics.accuracy_score(y_test, y_pred)

0.9242424242424242

In [239]:
metrics.confusion_matrix(y_test, y_pred)

array([[232,   4],
       [ 21,  73]])

In [240]:
metrics.recall_score(y_test, y_pred)

0.776595744680851

In [241]:
metrics.precision_score(y_test, y_pred)

0.948051948051948

#### Adaboost classifier

In [177]:
ada_gscv = GridSearchCV(AdaBoostClassifier(),cv=5,verbose=True,scoring='accuracy',n_jobs=-1,
             param_grid={'n_estimators':[200,300,400,600],'learning_rate':[0.1,.3,.5,.7,.9]})

ada_gscv.fit(x_train_res, y_train_res)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [200, 300, 400, 600], 'learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=True)

In [226]:
ada_gscv.best_score_

0.8277660324171953

In [209]:
# ada_gscv.cv_results_

In [227]:
ada_gscv.scoring

'accuracy'

In [228]:
y_pred = ada_gscv.best_estimator_.predict(X_test)

In [229]:
metrics.accuracy_score(y_test,y_pred)

0.7393939393939394

In [230]:
metrics.confusion_matrix(y_test,y_pred)

array([[204,  32],
       [ 54,  40]])

In [231]:
metrics.recall_score(y_test,y_pred)

0.425531914893617

In [232]:
metrics.precision_score(y_test,y_pred)

0.5555555555555556

#### Random Forest giving better results compared to Adaboost